In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon
from dbfread import DBF
import jenkspy
import json
from jenkspy import jenks_breaks

## Paso dos - Información censal

In [6]:
#RUTAS ENTRADA
ITER_2020_ruta = r"C:\Users\ADMIN\Desktop\Censos\ITER_NALCSV20.csv"
ITER_2010_ruta = r"C:\Users\ADMIN\Desktop\Censos\ITER_NALDBF10.dbf"

# ruta shp base
SHP_LOC_NAC_ruta = r"C:\Users\ADMIN\Desktop\Censos\shp\MG_2020_Integrado\conjunto_de_datos\00l.shp"

#ruta gpkg fin
MAPA_TIPO_ruta = r"C:\Users\ADMIN\Desktop\Ejercicio\Localidad\Loc_Tipologías_NAC.gpkg"

In [3]:
# PASO DOS - Preparación de información censal

# Función para cargar datos de archivo DBF y convertirlos a DataFrame
def load_dbf_to_dataframe(file_path):
    table = DBF(file_path)
    return pd.DataFrame(iter(table))

# Cargar archivos en un DataFrame
ITER_2020_df = pd.read_csv(ITER_2020_ruta, dtype={'ENTIDAD': str, 'MUN': str, 'LOC': str, 'AGEB': str, 'MZA': str})
ITER_2010_df = load_dbf_to_dataframe(ITER_2010_ruta)

# Homologación de ID rural - CVEGEO
def homologate_rural_id(row):
    return f"{row['ENTIDAD']}{row['MUN']}{row['LOC']}"

ITER_2020_df['CVEGEO'] = ITER_2020_df.apply(homologate_rural_id, axis=1)
ITER_2010_df['CVEGEO'] = ITER_2010_df.apply(homologate_rural_id, axis=1)

# reemplazar * con 1
def replace(dataframe):
    return dataframe.replace('*', '1')

ITER_2020_df = replace(ITER_2020_df)
ITER_2010_df = replace(ITER_2010_df)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_11792\4061011154.py:9: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  ITER_2020_df = pd.read_csv(ITER_2020_ruta, dtype={'ENTIDAD': str, 'MUN': str, 'LOC': str, 'AGEB': str, 'MZA': str})


In [14]:
# Leer shp base y convertir a gpkg
SHP_LOC_gdf = gpd.read_file(SHP_LOC_NAC_ruta)

# CREAR GPKG BASE DESDE gdf DE MANZANAS
SHP_LOC_gdf.to_file(MAPA_TIPO_ruta, layer='Mapa_Tipologias', driver='GPKG')

# Leer gpkg base
MAPA_TIPO_gdf = gpd.read_file(MAPA_TIPO_ruta, layer='Mapa_Tipologias')

# Filtrar censos
ITER_2020_df_F = ITER_2020_df[['CVEGEO', 'POBTOT', 'VIVTOT', 'VIVPAR_DES', 'VPH_1CUART', 'VPH_2CUART', 'VPH_3YMASC']]
ITER_2010_df_F = ITER_2010_df[['CVEGEO', 'POBTOT', 'VIVTOT', 'VIVPAR_DES', 'VPH_1CUART', 'VPH_2CUART', 'VPH_3YMASC']]

# Renombrar columnas de ITER_2020_df antes del merge para evitar conflictos
ITER_2020_df_renamed = ITER_2020_df_F.rename(columns={
    'POBTOT': 'POPTOT2020', 'VIVTOT': 'VIVTOT2020', 'VIVPAR_DES': 'VIVPAR_DES2020',
    'VPH_1CUART': 'VIV1C2020', 'VPH_2CUART': 'VIV2C2020', 'VPH_3YMASC': 'VIV3C2020'})
ITER_2010_df_renamed = ITER_2010_df_F.rename(columns={
    'POBTOT': 'POPTOT2010', 'VIVTOT': 'VIVTOT2010', 'VIVPAR_DES': 'VIVPAR_DES2010',
    'VPH_1CUART': 'VIV1C2010', 'VPH_2CUART': 'VIV2C2010', 'VPH_3YMASC': 'VIV3C2010'})

MAPA_TIPO_gdf['CVEGEO'] = MAPA_TIPO_gdf['CVEGEO'].astype(str)
ITER_2020_df_renamed['CVEGEO'] = ITER_2020_df_renamed['CVEGEO'].astype(str)
ITER_2010_df_renamed['CVEGEO'] = ITER_2010_df_renamed['CVEGEO'].astype(str)

# Unir campos de los DataFrames al GeoPackage base por CVEGEO
MAPA_TIPO_gdf1 = MAPA_TIPO_gdf.merge(
    ITER_2020_df_renamed[['CVEGEO', 'POPTOT2020', 'VIVTOT2020', 'VIVPAR_DES2020', 'VIV1C2020', 'VIV2C2020', 'VIV3C2020']], 
    on='CVEGEO', how='left')
MAPA_TIPO_gdf2 = MAPA_TIPO_gdf1.merge(
    ITER_2010_df_renamed[['CVEGEO', 'POPTOT2010', 'VIVTOT2010', 'VIVPAR_DES2010', 'VIV1C2010', 'VIV2C2010', 'VIV3C2010']], 
    on='CVEGEO', how='left')

# Reemplazar * con 1
def replace(dataframe):
    return dataframe.replace('*', '1')

# Limpiar *
MAPA_TIPO_gdf2_A = replace(MAPA_TIPO_gdf2)

# Reemplazar N/D
MAPA_TIPO_gdf2_B = MAPA_TIPO_gdf2_A.replace('N/D', np.nan)

# Actualizar valores sin información de "null" a 0
MAPA_TIPO_gdf2_C = MAPA_TIPO_gdf2_B.fillna(0)

# Convertir campos a enteros
fields_to_convert = ['POPTOT2010', 'POPTOT2020', 'VIVTOT2010', 'VIVTOT2020', 'VIVPAR_DES2010', 'VIVPAR_DES2020', 'VIV1C2010', 'VIV1C2020', 'VIV2C2010', 'VIV2C2020', 'VIV3C2010', 'VIV3C2020']
for field in fields_to_convert:
    MAPA_TIPO_gdf2_C[field] = MAPA_TIPO_gdf2_C[field].astype(int)

# Reordenar columnas
ordered_columns = [
    'CVEGEO', 'POPTOT2010', 'POPTOT2020', 'VIVTOT2010', 'VIVTOT2020', 'VIVPAR_DES2010', 'VIVPAR_DES2020',
    'VIV1C2010', 'VIV1C2020', 'VIV2C2010', 'VIV2C2020', 'VIV3C2010', 'VIV3C2020'
] + [col for col in MAPA_TIPO_gdf2_C.columns if col not in [
    'CVEGEO', 'POPTOT2010', 'POPTOT2020', 'VIVTOT2010', 'VIVTOT2020', 'VIVPAR_DES2010', 'VIVPAR_DES2020',
    'VIV1C2010', 'VIV1C2020', 'VIV2C2010', 'VIV2C2020', 'VIV3C2010', 'VIV3C2020'
]]

MAPA_TIPO_gdf2_C = MAPA_TIPO_gdf2_C[ordered_columns]

# Guardar el GeoPackage base actualizado
MAPA_TIPO_gdf2_C.to_file(MAPA_TIPO_ruta, layer='BASE_DATOS_NAC_LOC', driver="GPKG")

In [15]:
#ruta gpkg fin
MAPA_TIPO_ruta = r"C:\Users\ADMIN\Desktop\Ejercicio\Localidad\Loc_Tipologías_NAC.gpkg"
CVE_METRO_ruta = r"C:\Users\ADMIN\Desktop\Ejercicio\ClavesMetro.csv"

#AGRUPAR Y ASIGNAR FILTRO Y PROM POR ZM

#Subir archivo con CVE_METRO
CVE_METRO_df = pd.read_csv(CVE_METRO_ruta, dtype={'CVE_MET': str, 'CVEMUN':str})

# Leer la capa existente del GeoPackage base
MAPA_TIPO_gdf = gpd.read_file(MAPA_TIPO_ruta, layer='BASE_DATOS_NAC_LOC')

#Crear campo CVEMUN
def homologate_municipio_id(row):
    return f"{row['CVE_ENT']}{row['CVE_MUN']}"

MAPA_TIPO_gdf['CVEMUN'] = MAPA_TIPO_gdf.apply(homologate_municipio_id, axis=1)

In [16]:
#Añadir claves a GPKG
MAPA_TIPO_gdf = MAPA_TIPO_gdf.merge(CVE_METRO_df[['CVEMUN', 'CVE_MET', 'NOM_MET', 'TIPO_MET','TIPO_MUN']], on='CVEMUN', how='left')

#FILTRAR MUNICIPIOS ZONAS METROPOLITANAS
valores_filtro = CVE_METRO_df['CVEMUN'].unique()

MAPA_TIPO_gdf = MAPA_TIPO_gdf[(MAPA_TIPO_gdf['CVEMUN'].isin(valores_filtro))]

# Guardar el GeoPackage base actualizado
MAPA_TIPO_gdf.to_file(MAPA_TIPO_ruta, layer= 'BASE_DATOS_METRO_LOC', driver="GPKG")

In [17]:
# Leer la capa existente del GeoPackage base
MAPA_TIPO_gdf = gpd.read_file(MAPA_TIPO_ruta, layer='BASE_DATOS_METRO_LOC')

In [18]:
# Define el CRS proyectado que deseas utilizar
CRS_PROYECTADO = 'EPSG:6372' 

# Proyecta el GeoDataFrame a un CRS proyectado
MAPA_TIPO_gdf = MAPA_TIPO_gdf.to_crs(CRS_PROYECTADO)

# Creación de campos y llenado de información
MAPA_TIPO_gdf['area_ha'] = MAPA_TIPO_gdf.area / 10000
MAPA_TIPO_gdf['DESHABITACION'] = MAPA_TIPO_gdf['VIVPAR_DES2020'] / MAPA_TIPO_gdf['VIVTOT2020']

In [20]:
#resto de campos
MAPA_TIPO_gdf['DIFPOP'] = MAPA_TIPO_gdf['POPTOT2020'] - MAPA_TIPO_gdf['POPTOT2010']
MAPA_TIPO_gdf['DIFVIV'] = MAPA_TIPO_gdf['VIVTOT2020'] - MAPA_TIPO_gdf['VIVTOT2010']
MAPA_TIPO_gdf['RELPOP'] = MAPA_TIPO_gdf['DIFPOP'] / MAPA_TIPO_gdf['POPTOT2010']
MAPA_TIPO_gdf['RELVIV'] = MAPA_TIPO_gdf['DIFVIV'] / MAPA_TIPO_gdf['VIVTOT2010']
MAPA_TIPO_gdf['CVEPOP'] = np.where(MAPA_TIPO_gdf['RELPOP'] < -0.000000001, 'A', 'B')
MAPA_TIPO_gdf['CVEVIV'] = np.where(MAPA_TIPO_gdf['RELVIV'] < -.0703, '1',
                                   np.where((MAPA_TIPO_gdf['RELVIV'] >= -.0703) & (MAPA_TIPO_gdf['RELVIV'] <= .0703), 
                                            '2', '3'))
MAPA_TIPO_gdf['CVE'] = MAPA_TIPO_gdf['CVEPOP'] + MAPA_TIPO_gdf['CVEVIV'].astype(str)
conditions = [
    (MAPA_TIPO_gdf['POPTOT2020'] == 0),
    (MAPA_TIPO_gdf['POPTOT2010'] == 0) & (MAPA_TIPO_gdf['VIVTOT2010'] == 0) & (MAPA_TIPO_gdf['CVE'] == 'B3')
]
choices = ['DH', 'B3\'']
MAPA_TIPO_gdf['TIPOLOGIA'] = np.select(conditions, choices, default=MAPA_TIPO_gdf['CVE'])

MAPA_TIPO_gdf['DENS_POP20'] = MAPA_TIPO_gdf['POPTOT2020'] / MAPA_TIPO_gdf['area_ha']
MAPA_TIPO_gdf['DENS_POP10'] = MAPA_TIPO_gdf['POPTOT2010'] / MAPA_TIPO_gdf['area_ha']

MAPA_TIPO_gdf['DENS_VIV20'] = MAPA_TIPO_gdf['VIVTOT2020'] / MAPA_TIPO_gdf['area_ha']
MAPA_TIPO_gdf['DENS_VIV10'] = MAPA_TIPO_gdf['VIVTOT2010'] / MAPA_TIPO_gdf['area_ha']

MAPA_TIPO_gdf['CUARTOS2010'] = ((MAPA_TIPO_gdf['VIV1C2010']*1) + (MAPA_TIPO_gdf['VIV2C2010']*2) + (MAPA_TIPO_gdf['VIV3C2010']*4))
MAPA_TIPO_gdf['CUARTOS2020'] = ((MAPA_TIPO_gdf['VIV1C2020']*1) + (MAPA_TIPO_gdf['VIV2C2020']*2) + (MAPA_TIPO_gdf['VIV3C2020']*4))

MAPA_TIPO_gdf['CPERC2010'] = MAPA_TIPO_gdf['CUARTOS2010'] / MAPA_TIPO_gdf['POPTOT2010']
MAPA_TIPO_gdf['CPERC2020'] = MAPA_TIPO_gdf['CUARTOS2020'] / MAPA_TIPO_gdf['POPTOT2020']

MAPA_TIPO_gdf['HABXVIV2010'] = MAPA_TIPO_gdf['POPTOT2010'] / MAPA_TIPO_gdf['VIVTOT2010']
MAPA_TIPO_gdf['HABXVIV2020'] = MAPA_TIPO_gdf['POPTOT2020'] / MAPA_TIPO_gdf['VIVTOT2020']

def asignar_subtipologia(row):
    # Valores específicos para deshabitación
    rangos_deshabitacion_grupo = [0, .0703, .1416, .2812]  # Modifica estos valores según tus necesidades
    
# Asignar subtipología según los rangos definidos
    if rangos_deshabitacion_grupo[0] <= row['DESHABITACION'] < rangos_deshabitacion_grupo[1]:
        return 'a'
    elif rangos_deshabitacion_grupo[1] <= row['DESHABITACION'] < rangos_deshabitacion_grupo[2]:
        return 'b'
    elif rangos_deshabitacion_grupo[2] <= row['DESHABITACION'] < rangos_deshabitacion_grupo[3]:
        return 'c'
    elif row['DESHABITACION'] >= rangos_deshabitacion_grupo[3]:
        return 'd'
    else:
        return '0'  # Valor predeterminado

# Crear un nuevo campo 'R_DESHAB' y llenarlo con las subtipologías asignadas
MAPA_TIPO_gdf['R_DESHAB'] = MAPA_TIPO_gdf.apply(asignar_subtipologia, axis=1)

# Crear el campo SUBTIPOLOGÍA
MAPA_TIPO_gdf['SUBTIPOLOGÍA'] = MAPA_TIPO_gdf['TIPOLOGIA'].astype(str) + '-' + MAPA_TIPO_gdf['R_DESHAB'].astype(str)

# Guardar el GeoPackage con los nuevos campos
MAPA_TIPO_gdf.to_file(MAPA_TIPO_ruta, layer='Loc_Subtipologias', driver='GPKG')


# Resultados por Metrópoli

In [28]:
import pandas as pd
import numpy as np
import openpyxl
from openpyxl.styles import Font, PatternFill

# Columnas que se sumarán
columnas_a_sumar = ['POPTOT2010', 'POPTOT2020', 'VIVTOT2010', 'VIVTOT2020', 'VIVPAR_DES2010',
                    'VIVPAR_DES2020', 'VIV1C2010', 'VIV1C2020', 'VIV2C2010', 'VIV2C2020',
                    'VIV3C2010', 'VIV3C2020', 'area_ha']

# Sumar las columnas y agrupar por 'NOM_MET'
MAPA_TIPO_sumado = MAPA_TIPO_gdf.groupby(['NOM_MET', 'TIPO_MET'])[columnas_a_sumar].sum().reset_index()

# Contar las manzanas agrupadas
MAPA_TIPO_sumado['Localidades'] = MAPA_TIPO_gdf.groupby(['NOM_MET', 'TIPO_MET']).size().values

# Recalcular los campos
MAPA_TIPO_sumado['DIFPOP'] = MAPA_TIPO_sumado['POPTOT2020'] - MAPA_TIPO_sumado['POPTOT2010']
MAPA_TIPO_sumado['DIFVIV'] = MAPA_TIPO_sumado['VIVTOT2020'] - MAPA_TIPO_sumado['VIVTOT2010']
MAPA_TIPO_sumado['RELPOP'] = MAPA_TIPO_sumado['DIFPOP'] / MAPA_TIPO_sumado['POPTOT2010']
MAPA_TIPO_sumado['RELVIV'] = MAPA_TIPO_sumado['DIFVIV'] / MAPA_TIPO_sumado['VIVTOT2010']
MAPA_TIPO_sumado['CVEPOP'] = np.where(MAPA_TIPO_sumado['RELPOP'] < -0.000000001, 'A', 'B')
MAPA_TIPO_sumado['CVEVIV'] = np.where(MAPA_TIPO_sumado['RELVIV'] < -.0703, '1',
                                      np.where((MAPA_TIPO_sumado['RELVIV'] >= -.0703) & (MAPA_TIPO_sumado['RELVIV'] <= .0703), 
                                               '2', '3'))
MAPA_TIPO_sumado['CVE'] = MAPA_TIPO_sumado['CVEPOP'] + MAPA_TIPO_sumado['CVEVIV'].astype(str)

conditions = [
    (MAPA_TIPO_sumado['POPTOT2020'] == 0),
    (MAPA_TIPO_sumado['POPTOT2010'] == 0) & (MAPA_TIPO_sumado['VIVTOT2010'] == 0) & (MAPA_TIPO_sumado['CVE'] == 'B3')
]
choices = ['DH', 'B3\'']
MAPA_TIPO_sumado['TIPOLOGIA'] = np.select(conditions, choices, default=MAPA_TIPO_sumado['CVE'])

MAPA_TIPO_sumado['DENS_POP10'] = MAPA_TIPO_sumado['POPTOT2010'] / MAPA_TIPO_sumado['area_ha']
MAPA_TIPO_sumado['DENS_POP20'] = MAPA_TIPO_sumado['POPTOT2020'] / MAPA_TIPO_sumado['area_ha']
MAPA_TIPO_sumado['DIF_DENSPOP'] = MAPA_TIPO_sumado['DENS_POP20'] - MAPA_TIPO_sumado['DENS_POP10']

MAPA_TIPO_sumado['DENS_VIV10'] = MAPA_TIPO_sumado['VIVTOT2010'] / MAPA_TIPO_sumado['area_ha']
MAPA_TIPO_sumado['DENS_VIV20'] = MAPA_TIPO_sumado['VIVTOT2020'] / MAPA_TIPO_sumado['area_ha']
MAPA_TIPO_sumado['DIF_DENSVIV'] = MAPA_TIPO_sumado['DENS_VIV20'] - MAPA_TIPO_sumado['DENS_VIV10']

MAPA_TIPO_sumado['CUARTOS2010'] = ((MAPA_TIPO_sumado['VIV1C2010'] * 1) + (MAPA_TIPO_sumado['VIV2C2010'] * 2) + (MAPA_TIPO_sumado['VIV3C2010'] * 4))
MAPA_TIPO_sumado['CUARTOS2020'] = ((MAPA_TIPO_sumado['VIV1C2020'] * 1) + (MAPA_TIPO_sumado['VIV2C2020'] * 2) + (MAPA_TIPO_sumado['VIV3C2020'] * 4))

MAPA_TIPO_sumado['CPERC2010'] = MAPA_TIPO_sumado['CUARTOS2010'] / MAPA_TIPO_sumado['POPTOT2010']
MAPA_TIPO_sumado['CPERC2020'] = MAPA_TIPO_sumado['CUARTOS2020'] / MAPA_TIPO_sumado['POPTOT2020']
MAPA_TIPO_sumado['DIF_CPERC'] = MAPA_TIPO_sumado['CPERC2020'] - MAPA_TIPO_sumado['CPERC2010']

MAPA_TIPO_sumado['HABXVIV2010'] = MAPA_TIPO_sumado['POPTOT2010'] / MAPA_TIPO_sumado['VIVTOT2010']
MAPA_TIPO_sumado['HABXVIV2020'] = MAPA_TIPO_sumado['POPTOT2020'] / MAPA_TIPO_sumado['VIVTOT2020']
MAPA_TIPO_sumado['DIF_HABXVIV'] = MAPA_TIPO_sumado['HABXVIV2020'] - MAPA_TIPO_sumado['HABXVIV2010']


MAPA_TIPO_sumado['DESHABITACION_2010'] = MAPA_TIPO_sumado['VIVPAR_DES2010'] / MAPA_TIPO_sumado['VIVTOT2010']
MAPA_TIPO_sumado['DESHABITACION_2020'] = MAPA_TIPO_sumado['VIVPAR_DES2020'] / MAPA_TIPO_sumado['VIVTOT2020']
MAPA_TIPO_sumado['DIF_DESHAB'] = MAPA_TIPO_sumado['DESHABITACION_2020'] - MAPA_TIPO_sumado['DESHABITACION_2010']

# Calcular TASA_POP_MUN
MAPA_TIPO_sumado['TASA_POP'] = (((MAPA_TIPO_sumado['POPTOT2020'] / MAPA_TIPO_sumado['POPTOT2010']) ** (1/10)) - 1) * 100
MAPA_TIPO_sumado['TASA_VIV'] = (((MAPA_TIPO_sumado['VIVTOT2020'] / MAPA_TIPO_sumado['VIVTOT2010']) ** (1/10)) - 1) * 100

def asignar_subtipologia(row):
    # Valores específicos para deshabitación
    rangos_deshabitacion_grupo = [0, .0703, .1416, .2812]  # Modifica estos valores según tus necesidades
    
    # Asignar subtipología según los rangos definidos
    if rangos_deshabitacion_grupo[0] <= row['DESHABITACION_2020'] < rangos_deshabitacion_grupo[1]:
        return 'a'
    elif rangos_deshabitacion_grupo[1] <= row['DESHABITACION_2020'] < rangos_deshabitacion_grupo[2]:
        return 'b'
    elif rangos_deshabitacion_grupo[2] <= row['DESHABITACION_2020'] < rangos_deshabitacion_grupo[3]:
        return 'c'
    elif row['DESHABITACION_2020'] >= rangos_deshabitacion_grupo[3]:
        return 'd'
    else:
        return '0'  # Valor predeterminado

# Crear un nuevo campo 'R_DESHAB' y llenarlo con las subtipologías asignadas
MAPA_TIPO_sumado['R_DESHAB'] = MAPA_TIPO_sumado.apply(asignar_subtipologia, axis=1)

# Crear el campo SUBTIPOLOGÍA
MAPA_TIPO_sumado['SUBTIPOLOGÍA'] = MAPA_TIPO_sumado['TIPOLOGIA'].astype(str) + '-' + MAPA_TIPO_sumado['R_DESHAB'].astype(str)

# Redondear a 3 decimales
MAPA_TIPO_sumado = MAPA_TIPO_sumado.round(3)

# Guardar los datos en un archivo Excel
excel_path = 'C:/Users/ADMIN/Desktop/Ejercicio/Localidad/resultados_metropoli.xlsx'
MAPA_TIPO_sumado.to_excel(excel_path, index=False)

# Leer el archivo Excel
wb = openpyxl.load_workbook(excel_path)

# Seleccionar la hoja de cálculo
sheet = wb.active

# Ajustar el ancho de las columnas automáticamente
for col in sheet.columns:
    max_length = 0
    column = col[0].column_letter
    for cell in col:
        try:
            if len(str(cell.value)) > max_length:
                max_length = len(cell.value)
        except:
            pass
    adjusted_width = (max_length + 2) * 1.2
    sheet.column_dimensions[column].width = adjusted_width

# Aplicar color rojo a las celdas negativas en columnas con prefijo 'DIF_'
red_font = Font(color="FF0000")

for col in sheet.iter_cols(min_row=2, min_col=1, max_col=sheet.max_column):
    col_name = sheet.cell(row=1, column=col[0].column).value
    if col_name.startswith('DIF_'):
        for cell in col:
            if isinstance(cell.value, (int, float)) and cell.value < 0:
                cell.font = red_font

# Guardar los cambios en el archivo Excel
wb.save('C:/Users/ADMIN/Desktop/Ejercicio/Localidad/resultados_metropoli_v2.xlsx')

# Resultados por Municipio

In [29]:
import pandas as pd
import numpy as np
import openpyxl
from openpyxl.styles import Font, PatternFill

# Columnas que se sumarán
columnas_a_sumar = ['POPTOT2010', 'POPTOT2020', 'VIVTOT2010', 'VIVTOT2020', 'VIVPAR_DES2010',
                    'VIVPAR_DES2020', 'VIV1C2010', 'VIV1C2020', 'VIV2C2010', 'VIV2C2020',
                    'VIV3C2010', 'VIV3C2020', 'area_ha']

# Sumar las columnas y agrupar por 'NOM_MET'
MAPA_TIPO_sumado = MAPA_TIPO_gdf.groupby(['CVEMUN','NOM_MET', 'TIPO_MET'])[columnas_a_sumar].sum().reset_index()

# Contar las manzanas agrupadas
MAPA_TIPO_sumado['Localidades'] = MAPA_TIPO_gdf.groupby(['CVEMUN', 'NOM_MET', 'TIPO_MET']).size().values

# Recalcular los campos
MAPA_TIPO_sumado['DIFPOP'] = MAPA_TIPO_sumado['POPTOT2020'] - MAPA_TIPO_sumado['POPTOT2010']
MAPA_TIPO_sumado['DIFVIV'] = MAPA_TIPO_sumado['VIVTOT2020'] - MAPA_TIPO_sumado['VIVTOT2010']
MAPA_TIPO_sumado['RELPOP'] = MAPA_TIPO_sumado['DIFPOP'] / MAPA_TIPO_sumado['POPTOT2010']
MAPA_TIPO_sumado['RELVIV'] = MAPA_TIPO_sumado['DIFVIV'] / MAPA_TIPO_sumado['VIVTOT2010']
MAPA_TIPO_sumado['CVEPOP'] = np.where(MAPA_TIPO_sumado['RELPOP'] < -0.000000001, 'A', 'B')
MAPA_TIPO_sumado['CVEVIV'] = np.where(MAPA_TIPO_sumado['RELVIV'] < -.0703, '1',
                                      np.where((MAPA_TIPO_sumado['RELVIV'] >= -.0703) & (MAPA_TIPO_sumado['RELVIV'] <= .0703), 
                                               '2', '3'))
MAPA_TIPO_sumado['CVE'] = MAPA_TIPO_sumado['CVEPOP'] + MAPA_TIPO_sumado['CVEVIV'].astype(str)

conditions = [
    (MAPA_TIPO_sumado['POPTOT2020'] == 0),
    (MAPA_TIPO_sumado['POPTOT2010'] == 0) & (MAPA_TIPO_sumado['VIVTOT2010'] == 0) & (MAPA_TIPO_sumado['CVE'] == 'B3')
]
choices = ['DH', 'B3\'']
MAPA_TIPO_sumado['TIPOLOGIA'] = np.select(conditions, choices, default=MAPA_TIPO_sumado['CVE'])

MAPA_TIPO_sumado['DENS_POP10'] = MAPA_TIPO_sumado['POPTOT2010'] / MAPA_TIPO_sumado['area_ha']
MAPA_TIPO_sumado['DENS_POP20'] = MAPA_TIPO_sumado['POPTOT2020'] / MAPA_TIPO_sumado['area_ha']
MAPA_TIPO_sumado['DIF_DENSPOP'] = MAPA_TIPO_sumado['DENS_POP20'] - MAPA_TIPO_sumado['DENS_POP10']

MAPA_TIPO_sumado['DENS_VIV10'] = MAPA_TIPO_sumado['VIVTOT2010'] / MAPA_TIPO_sumado['area_ha']
MAPA_TIPO_sumado['DENS_VIV20'] = MAPA_TIPO_sumado['VIVTOT2020'] / MAPA_TIPO_sumado['area_ha']
MAPA_TIPO_sumado['DIF_DENSVIV'] = MAPA_TIPO_sumado['DENS_VIV20'] - MAPA_TIPO_sumado['DENS_VIV10']

MAPA_TIPO_sumado['CUARTOS2010'] = ((MAPA_TIPO_sumado['VIV1C2010'] * 1) + (MAPA_TIPO_sumado['VIV2C2010'] * 2) + (MAPA_TIPO_sumado['VIV3C2010'] * 4))
MAPA_TIPO_sumado['CUARTOS2020'] = ((MAPA_TIPO_sumado['VIV1C2020'] * 1) + (MAPA_TIPO_sumado['VIV2C2020'] * 2) + (MAPA_TIPO_sumado['VIV3C2020'] * 4))

MAPA_TIPO_sumado['CPERC2010'] = MAPA_TIPO_sumado['CUARTOS2010'] / MAPA_TIPO_sumado['POPTOT2010']
MAPA_TIPO_sumado['CPERC2020'] = MAPA_TIPO_sumado['CUARTOS2020'] / MAPA_TIPO_sumado['POPTOT2020']
MAPA_TIPO_sumado['DIF_CPERC'] = MAPA_TIPO_sumado['CPERC2020'] - MAPA_TIPO_sumado['CPERC2010']

MAPA_TIPO_sumado['HABXVIV2010'] = MAPA_TIPO_sumado['POPTOT2010'] / MAPA_TIPO_sumado['VIVTOT2010']
MAPA_TIPO_sumado['HABXVIV2020'] = MAPA_TIPO_sumado['POPTOT2020'] / MAPA_TIPO_sumado['VIVTOT2020']
MAPA_TIPO_sumado['DIF_HABXVIV'] = MAPA_TIPO_sumado['HABXVIV2020'] - MAPA_TIPO_sumado['HABXVIV2010']


MAPA_TIPO_sumado['DESHABITACION_2010'] = MAPA_TIPO_sumado['VIVPAR_DES2010'] / MAPA_TIPO_sumado['VIVTOT2010']
MAPA_TIPO_sumado['DESHABITACION_2020'] = MAPA_TIPO_sumado['VIVPAR_DES2020'] / MAPA_TIPO_sumado['VIVTOT2020']
MAPA_TIPO_sumado['DIF_DESHAB'] = MAPA_TIPO_sumado['DESHABITACION_2020'] - MAPA_TIPO_sumado['DESHABITACION_2010']

# Calcular TASA_POP_MUN
MAPA_TIPO_sumado['TASA_POP'] = (((MAPA_TIPO_sumado['POPTOT2020'] / MAPA_TIPO_sumado['POPTOT2010']) ** (1/10)) - 1) * 100
MAPA_TIPO_sumado['TASA_VIV'] = (((MAPA_TIPO_sumado['VIVTOT2020'] / MAPA_TIPO_sumado['VIVTOT2010']) ** (1/10)) - 1) * 100

def asignar_subtipologia(row):
    # Valores específicos para deshabitación
    rangos_deshabitacion_grupo = [0, .0703, .1416, .2812]  # Modifica estos valores según tus necesidades
    
    # Asignar subtipología según los rangos definidos
    if rangos_deshabitacion_grupo[0] <= row['DESHABITACION_2020'] < rangos_deshabitacion_grupo[1]:
        return 'a'
    elif rangos_deshabitacion_grupo[1] <= row['DESHABITACION_2020'] < rangos_deshabitacion_grupo[2]:
        return 'b'
    elif rangos_deshabitacion_grupo[2] <= row['DESHABITACION_2020'] < rangos_deshabitacion_grupo[3]:
        return 'c'
    elif row['DESHABITACION_2020'] >= rangos_deshabitacion_grupo[3]:
        return 'd'
    else:
        return '0'  # Valor predeterminado

# Crear un nuevo campo 'R_DESHAB' y llenarlo con las subtipologías asignadas
MAPA_TIPO_sumado['R_DESHAB'] = MAPA_TIPO_sumado.apply(asignar_subtipologia, axis=1)

# Crear el campo SUBTIPOLOGÍA
MAPA_TIPO_sumado['SUBTIPOLOGÍA'] = MAPA_TIPO_sumado['TIPOLOGIA'].astype(str) + '-' + MAPA_TIPO_sumado['R_DESHAB'].astype(str)

# Redondear a 3 decimales
MAPA_TIPO_sumado = MAPA_TIPO_sumado.round(3)

# Guardar los datos en un archivo Excel
excel_path = 'C:/Users/ADMIN/Desktop/Ejercicio/Localidad/resultados_municipio.xlsx'
MAPA_TIPO_sumado.to_excel(excel_path, index=False)

# Leer el archivo Excel
wb = openpyxl.load_workbook(excel_path)

# Seleccionar la hoja de cálculo
sheet = wb.active

# Ajustar el ancho de las columnas automáticamente
for col in sheet.columns:
    max_length = 0
    column = col[0].column_letter
    for cell in col:
        try:
            if len(str(cell.value)) > max_length:
                max_length = len(cell.value)
        except:
            pass
    adjusted_width = (max_length + 2) * 1.2
    sheet.column_dimensions[column].width = adjusted_width

# Aplicar color rojo a las celdas negativas en columnas con prefijo 'DIF_'
red_font = Font(color="FF0000")

for col in sheet.iter_cols(min_row=2, min_col=1, max_col=sheet.max_column):
    col_name = sheet.cell(row=1, column=col[0].column).value
    if col_name.startswith('DIF_'):
        for cell in col:
            if isinstance(cell.value, (int, float)) and cell.value < 0:
                cell.font = red_font

# Guardar los cambios en el archivo Excel
wb.save('C:/Users/ADMIN/Desktop/Ejercicio/Localidad/resultados_municipio_v2.xlsx')